<a href="https://colab.research.google.com/github/kentokura/ox_2x2_retrograde_analysis/blob/main/ox2x2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ノードの３状態:
  - 未発見
  - 未訪問
  - 既訪問

処理:
- 初期状態"____"をunsolvedに追加する。 # unsolvedに積まれているノードは未訪問.
- unsolvedが空になるまで以下を行う # BFS開始
  1. unsolvedから先頭のノード(current_node)を抽出する。
  1. 抽出したノード(current_node)をunsolvedから削除する。
  1. ノード(current_node)から辿れる次のノード(next_nodes)全てを調べる。 # 探索
    1. もし、next_nodeが未発見ならば # unsolved, solvedのいずれにもnext_nodeが存在しない
      1. そのノードを未訪問にする。 # unsolvedに追加
    1. そうではなく、発見済みならば
      1. 探索しない。
    1. next_node[previous_node]にcurrent_nodeを追加する。
    1. current_node[next_node]にnext_nodeを追加する。
  1. ノード（current_node）を既訪問にする。 # solvedに追加

1. solvedをプリントする。

# インプット

In [ ]:
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [212]:
# csvの読み込むためのモジュール
import pandas as pd
from pandas import DataFrame
import numpy as np
from tabulate import tabulate # pandasのdfをきれいに出力するためのモジュール

# 処理

In [201]:
# state を入力すると、次のstateのリストを出力する関数

def nextStates(state: str) -> list: 
  next_states = []
  player = "_"
  if state.count('o') <= state.count('x'):
    player = "o"
  else:
    player = "x"
  # next_nodeの作成
  for i, piece in enumerate(state):
    if piece == "_":
      next_state = list(state)
      next_state[i] = player
      next_states.append(''.join(next_state))
      # print(next_states)
  return next_states


# テスト
state = "ooo_"
nextStates(state)

# stateを入力すると、正規化したstateを返す関数
def normalization(state: str) -> str: 
    normalization_state = state
    if state in ["o___", "_o__", "__o_", "___o"]:
      normalization_state = "o___"
    elif state ==  ["ox__", "o_x_", "_o_x", "xo__", "_x_o", "__xo", "x_o_", "__ox"]:
      normalization_state ="ox__"
    elif state ==  ["o__x", "_ox_", "x_o_", "_x_o"]:
      normalization_state = "o__x"
    elif state ==  ["x_oo", "xo_o", "oxo_", "_xoo", "oox_", "_oxo", "oo_x", "o_ox"]:
      normalization_state = "oxo_"
    elif state ==  ["xoo_", "ox_o", "o_xo", "_oox"]:
      normalization_state = "ox_o"
    elif state ==  ["oxox", "xxoo", "xoxo", "ooxx"]:
      normalization_state = "oxox"
    elif state ==  ["oxxo", "xoox"]:
      normalization_state = "oxxo"

    return normalization_state


In [219]:
### BFSでゲーム木を作成するプログラム ### 

###

### 関数群ここから ###

# state を入力すると、次のstateのリストを出力する関数

def nextStates(state: str) -> list: 
  next_states = []
  player = "_"
  if state.count('o') <= state.count('x'):
    player = "o"
  else:
    player = "x"
  # next_nodeの作成
  for i, piece in enumerate(state):
    if piece == "_":
      next_state = list(state)
      next_state[i] = player
      next_states.append(''.join(next_state))
      # print(next_states)
  return next_states

# stateを入力すると、正規化したstateを返す関数
def normalization(state: str) -> str: 
    normalization_state = state
    if state in ["o___", "_o__", "__o_", "___o"]:
      normalization_state = "o___"
    elif state ==  ["ox__", "o_x_", "_o_x", "xo__", "_x_o", "__xo", "x_o_", "__ox"]:
      normalization_state ="ox__"
    elif state ==  ["o__x", "_ox_", "x_o_", "_x_o"]:
      normalization_state = "o__x"
    elif state ==  ["x_oo", "xo_o", "oxo_", "_xoo", "oox_", "_oxo", "oo_x", "o_ox"]:
      normalization_state = "oxo_"
    elif state ==  ["xoo_", "ox_o", "o_xo", "_oox"]:
      normalization_state = "ox_o"
    elif state ==  ["oxox", "xxoo", "xoxo", "ooxx"]:
      normalization_state = "oxox"
    elif state ==  ["oxxo", "xoox"]:
      normalization_state = "oxxo"

    return normalization_state

### 関数群ここまで

### 設定ここから ###
printFlag = False

### 設定ここまで ###

###mainここから

# unsolvedDf, solvedDfの初期化
if printFlag:
  print("===")
  print("プログラム開始")
  print("===")
  print()
  print("データを初期化します")
cols = ["PREVIOUS_STATES", "STATE", "NEXT_STATES"] #[前の状態list， 状態, 次の状態list]
df = pd.DataFrame(index=[], columns=cols)
df.set_index("STATE")
unsolvedDf = df
solvedDf = df
if printFlag:
  print("データを初期化しました")
  print()

# 初期状態"____"をunsolvedに追加する。unsolvedに積まれているノードは未訪問.
if printFlag:
  print("===")
  print("BFSの準備")
  print("===")
  print()
  print("初期状態をセットします")
init_state = "____"
previous_state = ""
unsolvedDf = unsolvedDf.append(pd.Series([[previous_state], init_state, "unsolved"], index=df.columns, name=init_state))
if printFlag:
  print("初期状態をセットしました") # 確認
  print("確認[UNSOLVED_DF]:") # 確認
  print(unsolvedDf) # 確認
  print() # 確認

# unsolvedが空になるまで以下を行う. BFS開始
if printFlag:
  print("===")
  print("BFSを開始します")
  print("===")
  print()
for _ in range(1000): # while len(unsolvedDf) > 0: # 開発のためにfor文にしている。
  # unsolvedDfから先頭のノードをpopする。
  if len(unsolvedDf) <= 0:
    break;
  current_node = unsolvedDf.iloc[0]  # 先頭のノード(current_node)を抽出。
  unsolvedDf.drop(unsolvedDf.index[0], inplace=True)  # 抽出したノードをunsolvedから削除。
  # 先頭のノード(current_node)から次のノード(next_nodes)を探索する。
  next_states = nextStates(current_node.STATE) # 次のノードの探索結果
  current_node.NEXT_STATES = next_states # current_nodeのNEXT_STATESに探索結果を反映
  # 探索した全ての状態について、以下を行う。
  if printFlag:
    print("unsolvedDfからpopされたノード'{}'の探索を行います".format(current_node.STATE))
  if len(next_states) <= 0:
    if printFlag:
      print("    探索結果: このノードは末端です")
  for next_state in next_states:
    # もし、next_nodeが未発見ならば # unsolved, solvedのいずれにもnext_nodeが存在しない
    if (next_state not in unsolvedDf.STATE.values) and (next_state not in solvedDf.STATE.values):
      if next_state == current_node.STATE: # 次のノードが自身と同一
        if printFlag:
          print("探索結果: 自身のノード'{}'と同一です".format(next_state))
        continue;
      else:
        if printFlag:
          print("    探索結果: 未発見のノード'{}'です".format(next_state))
        # T)そのノードを未訪問にする。 # unsolvedに追加
        previous_state = [current_node.STATE]
        next_node = pd.Series([previous_state, next_state, "unsolved"], index=df.columns, name=next_state) # next_nodeの作成
        unsolvedDf = unsolvedDf.append(next_node)
    else:  # F)そうではなく、発見済みならば
      if printFlag:
        print("    探索結果: 発見済みのノード'{}'です".format(next_state))
      #これを既に登録されていたノードのprevious_stateに追加する。
      previous_state = [current_node.STATE] 
      if next_state in unsolvedDf.STATE.values: # unsolvedDfに存在
        if printFlag:
          print("        これはunsolvedに存在しています")
        # unsolvedDf[unsolvedDf.STATE.values == next_state])にprevious_stateを追加する
        tmp = unsolvedDf.loc[next_state, "PREVIOUS_STATES"]
        tmp.append(previous_state[0])       
        unsolvedDf.loc[next_state, "PREVIOUS_STATES"] = tmp
      elif next_state in solvedDf.STATE.values:# solveDfに存在
        if printFlag:
          print("        これはsolvedに存在しています")
        # solvedDf[solvedDf.STATE.values == next_state])にprevious_stateを追加する
        tmp = solvedDf.loc[next_state, "PREVIOUS_STATES"]
        tmp.append(previous_state[0])       
        solvedDf.loc[next_state, "PREVIOUS_STATES"] = tmp
      else: # 何らかの理由で漏れた状態
        print("        エラー")
  # 現在のノード（current_node）をsolvedDfに追加する。solvedDfのノードは既訪問。 
  solvedDf = solvedDf.append(current_node)
if printFlag:
  print()
  print("BFSが終了しました")
  print()

# 結果確認
print("===")
print("結果確認")
print("===")
print()
print("確認[unsolvedDf]:")
print(tabulate(unsolvedDf, unsolvedDf.columns,tablefmt='github', showindex=True))
print()
print("確認[solvedDf]:")
print(tabulate(solvedDf, solvedDf.columns,tablefmt='github', showindex=True))
print()


### mainここまで

===
結果確認
===

確認[unsolvedDf]:
| PREVIOUS_STATES   | STATE   | NEXT_STATES   |
|-------------------|---------|---------------|

確認[solvedDf]:
|      | PREVIOUS_STATES   | STATE   | NEXT_STATES                      |
|------|-------------------|---------|----------------------------------|
| ____ | ['']              | ____    | ['o___', '_o__', '__o_', '___o'] |
| o___ | ['____']          | o___    | ['ox__', 'o_x_', 'o__x']         |
| _o__ | ['____']          | _o__    | ['xo__', '_ox_', '_o_x']         |
| __o_ | ['____']          | __o_    | ['x_o_', '_xo_', '__ox']         |
| ___o | ['____']          | ___o    | ['x__o', '_x_o', '__xo']         |
| ox__ | ['o___']          | ox__    | ['oxo_', 'ox_o']                 |
| o_x_ | ['o___']          | o_x_    | ['oox_', 'o_xo']                 |
| o__x | ['o___']          | o__x    | ['oo_x', 'o_ox']                 |
| xo__ | ['_o__']          | xo__    | ['xoo_', 'xo_o']                 |
| _ox_ | ['_o__']          | _ox_    | ['oox_

### 出力結果

|      | PREVIOUS_STATES   | STATE   | NEXT_STATES                      |
|------|-------------------|---------|----------------------------------|
| ____ | ['']              | ____    | ['o___', '_o__', '__o_', '___o'] |
| o___ | ['____']          | o___    | ['ox__', 'o_x_', 'o__x']         |
| _o__ | ['____']          | _o__    | ['xo__', '_ox_', '_o_x']         |
| __o_ | ['____']          | __o_    | ['x_o_', '_xo_', '__ox']         |
| ___o | ['____']          | ___o    | ['x__o', '_x_o', '__xo']         |
| ox__ | ['o___']          | ox__    | ['oxo_', 'ox_o']                 |
| o_x_ | ['o___']          | o_x_    | ['oox_', 'o_xo']                 |
| o__x | ['o___']          | o__x    | ['oo_x', 'o_ox']                 |
| xo__ | ['_o__']          | xo__    | ['xoo_', 'xo_o']                 |
| _ox_ | ['_o__']          | _ox_    | ['oox_', '_oxo']                 |
| _o_x | ['_o__']          | _o_x    | ['oo_x', '_oox']                 |
| x_o_ | ['__o_']          | x_o_    | ['xoo_', 'x_oo']                 |
| _xo_ | ['__o_']          | _xo_    | ['oxo_', '_xoo']                 |
| __ox | ['__o_']          | __ox    | ['o_ox', '_oox']                 |
| x__o | ['___o']          | x__o    | ['xo_o', 'x_oo']                 |
| _x_o | ['___o']          | _x_o    | ['ox_o', '_xoo']                 |
| __xo | ['___o']          | __xo    | ['o_xo', '_oxo']                 |
| oxo_ | ['ox__', '_xo_']  | oxo_    | ['oxox']                         |
| ox_o | ['ox__', '_x_o']  | ox_o    | ['oxxo']                         |
| oox_ | ['o_x_', '_ox_']  | oox_    | ['ooxx']                         |
| o_xo | ['o_x_', '__xo']  | o_xo    | ['oxxo']                         |
| oo_x | ['o__x', '_o_x']  | oo_x    | ['ooxx']                         |
| o_ox | ['o__x', '__ox']  | o_ox    | ['oxox']                         |
| xoo_ | ['xo__', 'x_o_']  | xoo_    | ['xoox']                         |
| xo_o | ['xo__', 'x__o']  | xo_o    | ['xoxo']                         |
| _oxo | ['_ox_', '__xo']  | _oxo    | ['xoxo']                         |
| _oox | ['_o_x', '__ox']  | _oox    | ['xoox']                         |
| x_oo | ['x_o_', 'x__o']  | x_oo    | ['xxoo']                         |
| _xoo | ['_xo_', '_x_o']  | _xoo    | ['xxoo']                         |
| oxox | ['oxo_', 'o_ox']  | oxox    | []                               |
| oxxo | ['ox_o', 'o_xo']  | oxxo    | []                               |
| ooxx | ['oox_', 'oo_x']  | ooxx    | []                               |
| xoox | ['xoo_', '_oox']  | xoox    | []                               |
| xoxo | ['xo_o', '_oxo']  | xoxo    | []                               |
| xxoo | ['x_oo', '_xoo']  | xxoo    | []                               |


# 出力

In [220]:
# solvedDfをox_outputという名前で書き出し
solvedDf.to_csv('/content/drive/My Drive/ox/workspace/ox_output.csv')

In [223]:
# ox_outputの確認

solvedDf = pd.read_csv(
    "/content/drive/My Drive/ox/workspace/ox_output.csv",
    index_col=0, # 最初の１行はデータ名。
    encoding="cp932" # windowsの追加文字に対応。おまじないだと思えば良い。
    )
print(solvedDf)

       PREVIOUS_STATES STATE                       NEXT_STATES
____              ['']  ____  ['o___', '_o__', '__o_', '___o']
o___          ['____']  o___          ['ox__', 'o_x_', 'o__x']
_o__          ['____']  _o__          ['xo__', '_ox_', '_o_x']
__o_          ['____']  __o_          ['x_o_', '_xo_', '__ox']
___o          ['____']  ___o          ['x__o', '_x_o', '__xo']
ox__          ['o___']  ox__                  ['oxo_', 'ox_o']
o_x_          ['o___']  o_x_                  ['oox_', 'o_xo']
o__x          ['o___']  o__x                  ['oo_x', 'o_ox']
xo__          ['_o__']  xo__                  ['xoo_', 'xo_o']
_ox_          ['_o__']  _ox_                  ['oox_', '_oxo']
_o_x          ['_o__']  _o_x                  ['oo_x', '_oox']
x_o_          ['__o_']  x_o_                  ['xoo_', 'x_oo']
_xo_          ['__o_']  _xo_                  ['oxo_', '_xoo']
__ox          ['__o_']  __ox                  ['o_ox', '_oox']
x__o          ['___o']  x__o                  ['xo_o', 

計画

- [x] 入力、処理、出力の雛形作成
- [x] ２ｘ２のoxゲームを実装

## 2x2

- 探索しよう。
初期状態とルールか。
- 状態をぶちこむと、次の状態を返す関数。
- 次の状態からｄｆを適切に変更。

In [170]:
# nodeの勝ち負け判定に使う。
# csvの読み込み
df = pd.read_csv(
    "/content/drive/My Drive/ox/workspace/ox.csv",
    index_col=0, # 最初の１行はデータ名。
    encoding="cp932" # windowsの追加文字に対応。おまじないだと思えば良い。
    )
print(df)
# print(df.query("id==0").to_numpy(copy=True).flatten())

    front_id  next_id    state
id                            
0        NaN        1  _,_,_,_
